In [1]:
import pandas as pd
import psycopg2

In [3]:
#p = [{'id': 'sMyAcaaxTRqIxL1pmqO_tw', 'user_id': '16778240', 'name': 'Jeffrey Kent Gillan', 'user_email': 'jgillan@arizona.edu', 'join_time': '2024-09-03T20:25:39Z', 'leave_time': '2024-09-03T23:02:08Z', 'duration': 9389, 'registrant_id': '', 'failover': False, 'status': 'in_meeting', 'groupId': ''}, {'id': '', 'user_id': '16797696', 'name': 'UA DataLab', 'user_email': '', 'join_time': '2024-09-03T20:27:43Z', 'leave_time': '2024-09-03T20:27:58Z', 'duration': 15, 'registrant_id': '', 'failover': False, 'status': 'in_waiting_room', 'groupId': ''}, {'id': '', 'user_id': '16798720', 'name': 'UA DataLab', 'user_email': '', 'join_time': '2024-09-03T20:27:59Z', 'leave_time': '2024-09-03T23:02:07Z', 'duration': 9248, 'registrant_id': '', 'failover': False, 'status': 'in_meeting', 'groupId': ''}, {'id': '2M9An2omS9KPM8Rxz2lwfw', 'user_id': '16799744', 'name': 'Michele Cosi', 'user_email': 'cosi@arizona.edu', 'join_time': '2024-09-03T20:31:41Z', 'leave_time': '2024-09-03T23:02:08Z', 'duration': 9027, 'registrant_id': '', 'failover': False, 'status': 'in_meeting', 'groupId': ''}, {'id': '', 'user_id': '16819200', 'name': 'Sandeep Kulkarni', 'user_email': '', 'join_time': '2024-09-03T20:52:29Z', 'leave_time': '2024-09-03T20:53:13Z', 'duration': 44, 'registrant_id': '', 'failover': False, 'status': 'in_waiting_room', 'groupId': ''}, {'id': 'yXxO1qhiQkmJy4b80J8xAQ', 'user_id': '16820224', 'name': 'Carlos Lizárraga', 'user_email': 'clizarraga@arizona.edu', 'join_time': '2024-09-03T20:53:03Z', 'leave_time': '2024-09-03T21:27:21Z', 'duration': 2058, 'registrant_id': '', 'failover': False, 'status': 'in_meeting', 'groupId': ''}, {'id': '', 'user_id': '16821248', 'name': 'Sandeep Kulkarni', 'user_email': '', 'join_time': '2024-09-03T20:53:13Z', 'leave_time': '2024-09-03T21:53:31Z', 'duration': 3618, 'registrant_id': '', 'failover': False, 'status': 'in_meeting', 'groupId': ''}, {'id': 'uoTA-PPPScmSAMRObdoQig', 'user_id': '16822272', 'name': 'Tyson Swetnam', 'user_email': 'tswetnam@arizona.edu', 'join_time': '2024-09-03T20:53:51Z', 'leave_time': '2024-09-03T20:54:59Z', 'duration': 68, 'registrant_id': '', 'failover': False, 'status': 'in_waiting_room', 'groupId': ''}, {'id': '', 'user_id': '16823296', 'name': 'Eduardo Martinez', 'user_email': '', 'join_time': '2024-09-03T20:54:13Z', 'leave_time': '2024-09-03T20:55:04Z', 'duration': 51, 'registrant_id': '', 'failover': False, 'status': 'in_waiting_room', 'groupId': ''}, {'id': 'uoTA-PPPScmSAMRObdoQig', 'user_id': '16824320', 'name': 'Tyson Swetnam', 'user_email': 'tswetnam@arizona.edu', 'join_time': '2024-09-03T20:55:00Z', 'leave_time': '2024-09-03T21:53:19Z', 'duration': 3499, 'registrant_id': '', 'failover': False, 'status': 'in_meeting', 'groupId': ''}, {'id': '', 'user_id': '16825344', 'name': 'Eduardo Martinez', 'user_email': '', 'join_time': '2024-09-03T20:55:05Z', 'leave_time': '2024-09-03T21:53:24Z', 'duration': 3499, 'registrant_id': '', 'failover': False, 'status': 'in_meeting', 'groupId': ''}, {'id': 'qGebtr02SM-Q6CvMjYvn2A', 'user_id': '16826368', 'name': 'Dessa', 'user_email': 'dakeys@arizona.edu', 'join_time': '2024-09-03T20:56:03Z', 'leave_time': '2024-09-03T20:56:10Z', 'duration': 7, 'registrant_id': '', 'failover': False, 'status': 'in_waiting_room', 'groupId': 'h7f4ijsPQFGoJNQT3CfhrA'}, {'id': 'qGebtr02SM-Q6CvMjYvn2A', 'user_id': '16827392', 'name': 'Dessa', 'user_email': 'dakeys@arizona.edu', 'join_time': '2024-09-03T20:56:11Z', 'leave_time': '2024-09-03T21:53:38Z', 'duration': 3447, 'registrant_id': '', 'failover': False, 'status': 'in_meeting', 'groupId': 'h7f4ijsPQFGoJNQT3CfhrA'}, {'id': 'NsbTcPOoT_qDdx3tDdPQ8w', 'user_id': '16830464', 'name': 'Zach Winston', 'user_email': 'winston@arizona.edu', 'join_time': '2024-09-03T20:56:16Z', 'leave_time': '2024-09-03T20:56:27Z', 'duration': 11, 'registrant_id': '', 'failover': False, 'status': 'in_waiting_room', 'groupId': 'o4RhpwgjQNuSlfp_BzXIog'}, {'id': 'NsbTcPOoT_qDdx3tDdPQ8w', 'user_id': '16831488', 'name': 'Zach Winston', 'user_email': 'winston@arizona.edu', 'join_time': '2024-09-03T20:56:27Z', 'leave_time': '2024-09-03T21:24:20Z', 'duration': 1673, 'registrant_id': '', 'failover': False, 'status': 'in_meeting', 'groupId': 'o4RhpwgjQNuSlfp_BzXIog'}, {'id': '', 'user_id': '16832512', 'name': 'Yash Vijay Salokhe', 'user_email': '', 'join_time': '2024-09-03T20:57:25Z', 'leave_time': '2024-09-03T20:57:33Z', 'duration': 8, 'registrant_id': '', 'failover': False, 'status': 'in_waiting_room', 'groupId': ''}, {'id': '', 'user_id': '16833536', 'name': 'Yash Vijay Salokhe', 'user_email': '', 'join_time': '2024-09-03T20:57:33Z', 'leave_time': '2024-09-03T21:52:38Z', 'duration': 3305, 'registrant_id': '', 'failover': False, 'status': 'in_meeting', 'groupId': ''}, {'id': '', 'user_id': '16834560', 'name': 'Anirudh K', 'user_email': '', 'join_time': '2024-09-03T20:57:50Z', 'leave_time': '2024-09-03T20:58:20Z', 'duration': 30, 'registrant_id': '', 'failover': False, 'status': 'in_waiting_room', 'groupId': ''}, {'id': 'hBhR83maRx2z2O9ymxBrkw', 'user_id': '16835584', 'name': 'SAURABH KAUSHIK', 'user_email': 'skaushik@arizona.edu', 'join_time': '2024-09-03T20:58:07Z', 'leave_time': '2024-09-03T20:58:37Z', 'duration': 30, 'registrant_id': '', 'failover': False, 'status': 'in_waiting_room', 'groupId': ''}, {'id': '', 'user_id': '16836608', 'name': 'Dustin Curley', 'user_email': '', 'join_time': '2024-09-03T20:58:25Z', 'leave_time': '2024-09-03T20:58:39Z', 'duration': 14, 'registrant_id': '', 'failover': False, 'status': 'in_waiting_room', 'groupId': ''}, {'id': 'hBhR83maRx2z2O9ymxBrkw', 'user_id': '16837632', 'name': 'SAURABH KAUSHIK', 'user_email': 'skaushik@arizona.edu', 'join_time': '2024-09-03T20:58:38Z', 'leave_time': '2024-09-03T21:53:18Z', 'duration': 3280, 'registrant_id': '', 'failover': False, 'status': 'in_meeting', 'groupId': ''}, {'id': '', 'user_id': '16838656', 'name': 'Dustin Curley', 'user_email': '', 'join_time': '2024-09-03T20:58:39Z', 'leave_time': '2024-09-03T21:53:23Z', 'duration': 3284, 'registrant_id': '', 'failover': False, 'status': 'in_meeting', 'groupId': ''}, {'id': 'O89dEpRiSzKAnNR_ltqaCg', 'user_id': '16839680', 'name': 'Eduardo Jimenez', 'user_email': 'eduardojh@arizona.edu', 'join_time': '2024-09-03T20:58:51Z', 'leave_time': '2024-09-03T20:58:59Z', 'duration': 8, 'registrant_id': '', 'failover': False, 'status': 'in_waiting_room', 'groupId': 'o4RhpwgjQNuSlfp_BzXIog'}, {'id': 'Qe66mrfFQPqBc8BYDJtUTA', 'user_id': '16840704', 'name': 'Matthew Hall Moelling', 'user_email': 'mhmoelling@arizona.edu', 'join_time': '2024-09-03T20:58:54Z', 'leave_time': '2024-09-03T20:59:04Z', 'duration': 10, 'registrant_id': '', 'failover': False, 'status': 'in_waiting_room', 'groupId': ''}, {'id': 'O89dEpRiSzKAnNR_ltqaCg', 'user_id': '16841728', 'name': 'Eduardo Jimenez', 'user_email': 'eduardojh@arizona.edu', 'join_time': '2024-09-03T20:58:59Z', 'leave_time': '2024-09-03T21:53:53Z', 'duration': 3294, 'registrant_id': '', 'failover': False, 'status': 'in_meeting', 'groupId': 'o4RhpwgjQNuSlfp_BzXIog'}, {'id': 'Qe66mrfFQPqBc8BYDJtUTA', 'user_id': '16842752', 'name': 'Matthew Hall Moelling', 'user_email': 'mhmoelling@arizona.edu', 'join_time': '2024-09-03T20:59:05Z', 'leave_time': '2024-09-03T21:00:45Z', 'duration': 100, 'registrant_id': '', 'failover': False, 'status': 'in_meeting', 'groupId': ''}, {'id': '3pVAbp9aSaa7yNhvCKFq8w', 'user_id': '16843776', 'name': 'Andrew Corrales', 'user_email': 'andrewicorrales@arizona.edu', 'join_time': '2024-09-03T20:59:19Z', 'leave_time': '2024-09-03T20:59:37Z', 'duration': 18, 'registrant_id': '', 'failover': False, 'status': 'in_waiting_room', 'groupId': 'o4RhpwgjQNuSlfp_BzXIog'}, {'id': '5Shc802_TWyERUEchqUBhw', 'user_id': '16844800', 'name': 'Jeremy Weiss', 'user_email': 'jlweiss@arizona.edu', 'join_time': '2024-09-03T20:59:38Z', 'leave_time': '2024-09-03T20:59:45Z', 'duration': 7, 'registrant_id': '', 'failover': False, 'status': 'in_waiting_room', 'groupId': 'o4RhpwgjQNuSlfp_BzXIog'}, {'id': '3pVAbp9aSaa7yNhvCKFq8w', 'user_id': '16845824', 'name': 'Andrew Corrales', 'user_email': 'andrewicorrales@arizona.edu', 'join_time': '2024-09-03T20:59:38Z', 'leave_time': '2024-09-03T21:53:26Z', 'duration': 3228, 'registrant_id': '', 'failover': False, 'status': 'in_meeting', 'groupId': 'o4RhpwgjQNuSlfp_BzXIog'}, {'id': '5Shc802_TWyERUEchqUBhw', 'user_id': '16846848', 'name': 'Jeremy Weiss', 'user_email': 'jlweiss@arizona.edu', 'join_time': '2024-09-03T20:59:45Z', 'leave_time': '2024-09-03T21:53:20Z', 'duration': 3215, 'registrant_id': '', 'failover': False, 'status': 'in_meeting', 'groupId': 'o4RhpwgjQNuSlfp_BzXIog'}]
p = [{'id': 'vwMyYuolSe-jzaXoRXkOEQ', 'user_id': '16778240', 'name': 'Megh Krishnaswamy', 'user_email': 'mkrishnaswamy@arizona.edu', 'join_time': '2024-09-05T20:54:29Z', 'leave_time': '2024-09-05T22:51:35Z', 'duration': 7026, 'registrant_id': '', 'failover': False, 'status': 'in_meeting', 'groupId': ''}, {'id': 'Wg8f7A1CSZWFuyvumeq5Hw', 'user_id': '33555456', 'name': 'P. Mithün, PhD. (Dr. Mithün)', 'user_email': 'mithunpaul@arizona.edu', 'join_time': '2024-09-05T20:54:32Z', 'leave_time': '2024-09-05T22:51:35Z', 'duration': 7023, 'registrant_id': '', 'failover': False, 'status': 'in_meeting', 'groupId': ''}, {'id': '', 'user_id': '16797696', 'name': 'Prajwal Sathyanarayana', 'user_email': '', 'join_time': '2024-09-05T20:54:59Z', 'leave_time': '2024-09-05T20:55:14Z', 'duration': 15, 'registrant_id': '', 'failover': False, 'status': 'in_waiting_room', 'groupId': ''}, {'id': '', 'user_id': '16798720', 'name': 'Prajwal Sathyanarayana', 'user_email': '', 'join_time': '2024-09-05T20:55:14Z', 'leave_time': '2024-09-05T21:55:47Z', 'duration': 3633, 'registrant_id': '', 'failover': False, 'status': 'in_meeting', 'groupId': ''}, {'id': 'CUb3ZbjESTulYKNnvt8PAQ', 'user_id': '16799744', 'name': 'Enrique Noriega', 'user_email': 'enoriega@arizona.edu', 'join_time': '2024-09-05T20:56:20Z', 'leave_time': '2024-09-05T21:58:56Z', 'duration': 3756, 'registrant_id': '', 'failover': False, 'status': 'in_meeting', 'groupId': ''}, {'id': '', 'user_id': '16800768', 'name': 'Garrett Ryan Smith', 'user_email': '', 'join_time': '2024-09-05T20:56:46Z', 'leave_time': '2024-09-05T20:56:50Z', 'duration': 4, 'registrant_id': '', 'failover': False, 'status': 'in_waiting_room', 'groupId': ''}, {'id': '', 'user_id': '16801792', 'name': 'Garrett Ryan Smith', 'user_email': '', 'join_time': '2024-09-05T20:56:50Z', 'leave_time': '2024-09-05T21:57:09Z', 'duration': 3619, 'registrant_id': '', 'failover': False, 'status': 'in_meeting', 'groupId': ''}, {'id': '1aSJKnYxQgCdZJzFU1XsKg', 'user_id': '16802816', 'name': 'Mike B.', 'user_email': 'mikebenson@arizona.edu', 'join_time': '2024-09-05T21:00:00Z', 'leave_time': '2024-09-05T21:00:13Z', 'duration': 13, 'registrant_id': '', 'failover': False, 'status': 'in_waiting_room', 'groupId': ''}, {'id': '1aSJKnYxQgCdZJzFU1XsKg', 'user_id': '16803840', 'name': 'Mike B.', 'user_email': 'mikebenson@arizona.edu', 'join_time': '2024-09-05T21:00:13Z', 'leave_time': '2024-09-05T22:50:12Z', 'duration': 6599, 'registrant_id': '', 'failover': False, 'status': 'in_meeting', 'groupId': ''}, {'id': 'cwg48vWuSXybIJIbAzR5zQ', 'user_id': '16804864', 'name': 'Bryan Heidorn', 'user_email': 'heidorn@arizona.edu', 'join_time': '2024-09-05T21:00:16Z', 'leave_time': '2024-09-05T21:00:21Z', 'duration': 5, 'registrant_id': '', 'failover': False, 'status': 'in_waiting_room', 'groupId': ''}, {'id': 'cwg48vWuSXybIJIbAzR5zQ', 'user_id': '16805888', 'name': 'Bryan Heidorn', 'user_email': 'heidorn@arizona.edu', 'join_time': '2024-09-05T21:00:21Z', 'leave_time': '2024-09-05T21:58:56Z', 'duration': 3515, 'registrant_id': '', 'failover': False, 'status': 'in_meeting', 'groupId': ''}, {'id': '', 'user_id': '16806912', 'name': 'Keith Alcock', 'user_email': '', 'join_time': '2024-09-05T21:00:48Z', 'leave_time': '2024-09-05T21:01:09Z', 'duration': 21, 'registrant_id': '', 'failover': False, 'status': 'in_waiting_room', 'groupId': ''}, {'id': '', 'user_id': '16807936', 'name': 'Keith Alcock', 'user_email': '', 'join_time': '2024-09-05T21:01:10Z', 'leave_time': '2024-09-05T22:50:43Z', 'duration': 6573, 'registrant_id': '', 'failover': False, 'status': 'in_meeting', 'groupId': ''}, {'id': '_lE_dT9vRAqgPi4f64Qaiw', 'user_id': '16808960', 'name': 'Maliaca Oxnam | U of A™', 'user_email': 'maliaca@arizona.edu', 'join_time': '2024-09-05T21:02:06Z', 'leave_time': '2024-09-05T21:02:10Z', 'duration': 4, 'registrant_id': '', 'failover': False, 'status': 'in_waiting_room', 'groupId': ''}, {'id': '_lE_dT9vRAqgPi4f64Qaiw', 'user_id': '16809984', 'name': 'Maliaca Oxnam | U of A™', 'user_email': 'maliaca@arizona.edu', 'join_time': '2024-09-05T21:02:10Z', 'leave_time': '2024-09-05T22:48:35Z', 'duration': 6385, 'registrant_id': '', 'failover': False, 'status': 'in_meeting', 'groupId': ''}, {'id': '', 'user_id': '16811008', 'name': 'Sahil Makhijani', 'user_email': '', 'join_time': '2024-09-05T21:03:30Z', 'leave_time': '2024-09-05T21:03:54Z', 'duration': 24, 'registrant_id': '', 'failover': False, 'status': 'in_waiting_room', 'groupId': ''}, {'id': '', 'user_id': '16812032', 'name': 'Sahil Makhijani', 'user_email': '', 'join_time': '2024-09-05T21:03:54Z', 'leave_time': '2024-09-05T22:51:22Z', 'duration': 6448, 'registrant_id': '', 'failover': False, 'status': 'in_meeting', 'groupId': ''}, {'id': 'KtCe_4ErTHKzXrEKD6Za9A', 'user_id': '16813056', 'name': 'Shashank Yadav', 'user_email': 'shashank@arizona.edu', 'join_time': '2024-09-05T21:05:15Z', 'leave_time': '2024-09-05T21:05:19Z', 'duration': 4, 'registrant_id': '', 'failover': False, 'status': 'in_waiting_room', 'groupId': ''}, {'id': 'KtCe_4ErTHKzXrEKD6Za9A', 'user_id': '16814080', 'name': 'Shashank Yadav', 'user_email': 'shashank@arizona.edu', 'join_time': '2024-09-05T21:05:20Z', 'leave_time': '2024-09-05T21:58:38Z', 'duration': 3198, 'registrant_id': '', 'failover': False, 'status': 'in_meeting', 'groupId': ''}, {'id': '', 'user_id': '16815104', 'name': 'Yash Vijay Salokhe', 'user_email': '', 'join_time': '2024-09-05T21:05:55Z', 'leave_time': '2024-09-05T21:06:01Z', 'duration': 6, 'registrant_id': '', 'failover': False, 'status': 'in_waiting_room', 'groupId': ''}, {'id': '', 'user_id': '16816128', 'name': 'Sandeep Kulkarni', 'user_email': '', 'join_time': '2024-09-05T21:06:00Z', 'leave_time': '2024-09-05T21:06:12Z', 'duration': 12, 'registrant_id': '', 'failover': False, 'status': 'in_waiting_room', 'groupId': ''}, {'id': '', 'user_id': '16817152', 'name': 'Yash Vijay Salokhe', 'user_email': '', 'join_time': '2024-09-05T21:06:01Z', 'leave_time': '2024-09-05T22:33:47Z', 'duration': 5266, 'registrant_id': '', 'failover': False, 'status': 'in_meeting', 'groupId': ''}, {'id': '', 'user_id': '16818176', 'name': 'Sandeep Kulkarni', 'user_email': '', 'join_time': '2024-09-05T21:06:13Z', 'leave_time': '2024-09-05T21:59:36Z', 'duration': 3203, 'registrant_id': '', 'failover': False, 'status': 'in_meeting', 'groupId': ''}, {'id': 'uG19mExtQGunQO5OEVfmNw', 'user_id': '16819200', 'name': 'Maiya Block Ngaybe', 'user_email': 'mgblock@arizona.edu', 'join_time': '2024-09-05T21:09:16Z', 'leave_time': '2024-09-05T21:09:22Z', 'duration': 6, 'registrant_id': '', 'failover': False, 'status': 'in_waiting_room', 'groupId': 'h7f4ijsPQFGoJNQT3CfhrA'}, {'id': 'uG19mExtQGunQO5OEVfmNw', 'user_id': '16820224', 'name': 'Maiya Block Ngaybe', 'user_email': 'mgblock@arizona.edu', 'join_time': '2024-09-05T21:09:23Z', 'leave_time': '2024-09-05T22:04:54Z', 'duration': 3331, 'registrant_id': '', 'failover': False, 'status': 'in_meeting', 'groupId': 'h7f4ijsPQFGoJNQT3CfhrA'}, {'id': 'NRq3f_DnRmy5P6nfshT0wg', 'user_id': '16821248', 'name': 'Ran Wei', 'user_email': 'ranwei@arizona.edu', 'join_time': '2024-09-05T21:11:26Z', 'leave_time': '2024-09-05T21:11:30Z', 'duration': 4, 'registrant_id': '', 'failover': False, 'status': 'in_waiting_room', 'groupId': 'h7f4ijsPQFGoJNQT3CfhrA'}, {'id': 'NRq3f_DnRmy5P6nfshT0wg', 'user_id': '16822272', 'name': 'Ran Wei', 'user_email': 'ranwei@arizona.edu', 'join_time': '2024-09-05T21:11:30Z', 'leave_time': '2024-09-05T21:58:19Z', 'duration': 2809, 'registrant_id': '', 'failover': False, 'status': 'in_meeting', 'groupId': 'h7f4ijsPQFGoJNQT3CfhrA'}, {'id': '', 'user_id': '16823296', 'name': 'Devashree Pawar', 'user_email': '', 'join_time': '2024-09-05T21:18:37Z', 'leave_time': '2024-09-05T21:18:47Z', 'duration': 10, 'registrant_id': '', 'failover': False, 'status': 'in_waiting_room', 'groupId': ''}, {'id': '', 'user_id': '16824320', 'name': 'Devashree Pawar', 'user_email': '', 'join_time': '2024-09-05T21:18:47Z', 'leave_time': '2024-09-05T21:59:50Z', 'duration': 2463, 'registrant_id': '', 'failover': False, 'status': 'in_meeting', 'groupId': ''}, {'id': '0A_sJr7mTx2-x8ckfg4uVw', 'user_id': '16825344', 'name': 'Oscar Daniel Moreno', 'user_email': 'oscardmoreno@arizona.edu', 'join_time': '2024-09-05T21:52:23Z', 'leave_time': '2024-09-05T21:52:36Z', 'duration': 13, 'registrant_id': '', 'failover': False, 'status': 'in_waiting_room', 'groupId': ''}]
len(p)

30

In [6]:
conn = psycopg2.connect(database = "DataLab", 
                            user = "postgres", 
                            host= 'localhost',
                            password = "",
                            port = 5432)
cur = conn.cursor()

cur.execute("""SELECT ElementValue FROM programvariables WHERE ElementName = 'qualtrics_api_token'""")
tokenList = cur.fetchone()
conn.commit()
apiKey = tokenList[0]
apiKey


'egl9Nc1pKnEswZ1hUMfXjjNJ50wTP9LhBHcJa4XF'

In [4]:
conn = psycopg2.connect(database = "DataLab", 
                            user = "postgres", 
                            host= 'localhost',
                            password = "",
                            port = 5432)
cur = conn.cursor()

cur.execute("""
            SELECT * FROM registreeworkshops WHERE checkedin = True AND registered = False;
            """)
conn.commit()
rows = cur.fetchall()

cur.close()
conn.close()

In [6]:
rando = pd.DataFrame(data=rows, columns=['RegID', 'WorkshopID', 'Registered', 'CheckedIn'])

In [10]:
conn = psycopg2.connect(database = "DataLab", 
                            user = "postgres", 
                            host= 'localhost',
                            password = "",
                            port = 5432)
cur = conn.cursor()

emails = []

for _,row in rando.iterrows():
        cur.execute("""
            SELECT Email FROM registreeInfo WHERE RegID = %s;
            """, (row.iloc[0],))
        conn.commit()
        rows = cur.fetchone()
        emails.append(rows[0])


cur.close()
conn.close()

In [19]:
conn = psycopg2.connect(database = "DataLab", 
                            user = "postgres", 
                            host= 'localhost',
                            password = "",
                            port = 5432)
cur = conn.cursor()
cur.execute("""
                SELECT workshops.WorkshopID, workshops.SeriesID, series.ZoomMeetingID, series.StartTime, series.EndTime FROM workshops
                JOIN series on workshops.SeriesID = series.SeriesID
                WHERE workshops.Workshopdate = now()::date;
                """)
workshopList = cur.fetchall()
conn.commit()

In [20]:
series = pd.DataFrame(data=workshopList, columns=['WorkshopID', 'SeriesID', 'ZoomID', 'StartTime', 'EndTime'])

In [2]:
import psycopg2
from datetime import datetime

In [6]:
time = datetime(2024, 4, 1, 14, 1, 30)
conn = psycopg2.connect(database = "DataLab", 
                            user = "postgres", 
                            host= 'localhost',
                            password = "",
                            port = 5432)
cur = conn.cursor()
cur.execute("""SELECT DISTINCT SeriesID FROM workshops 
            WHERE workshopDate = %s::date
            """, (time,))
match = cur.fetchall()
conn.commit()
cur.close()
conn.close()

In [8]:
import psycopg2
import pandas as pd

conn = psycopg2.connect(
    database="DataLab",
    user="postgres",
    host='localhost',
    password="",
    port=5432
)

# Create a cursor object
cur = conn.cursor()

# Read the CSV file containing the names
names_df = pd.read_csv("./Unregistered.csv")

# Initialize an empty DataFrame to store the results
results_df = pd.DataFrame(columns=['First Name', 'Last Name', 'Email'])

# Iterate over each row in the DataFrame
for _, row in names_df.iterrows():
    first_name = row['First Name']
    last_name = row['Last Name']
    email = row['Email']

    # Execute the SELECT query for the current name
    cur.execute("""
        SELECT ri.FirstName, ri.LastName, ri.Email
        FROM RegistreeInfo as ri
        JOIN RegistreeWorkshops as rw ON ri.RegID = rw.RegID
        WHERE (rw.CheckedIn = TRUE OR rw.Registered = TRUE)
        AND (ri.FirstName = %s AND ri.LastName = %s)
    """, (first_name, last_name))

    # Fetch all rows from the result set
    rows = cur.fetchall()

    # Create a DataFrame from the fetched rows
    df = pd.DataFrame(rows, columns=['First Name', 'Last Name', 'Email'])

    # Concatenate the DataFrame to the results DataFrame
    results_df = pd.concat([results_df, df])

# Write the results DataFrame to a CSV file
results_df.to_csv("./Duplicates.csv", index=False)

# Close the cursor and connection
cur.close()
conn.close()


In [10]:
from datetime import datetime, timezone, timedelta

def are_dates_equal(arizona_date_str, utc_date_str):
    arizona_date = datetime.strptime(arizona_date_str, "%Y-%m-%d")
    utc_date = datetime.strptime(utc_date_str, "%Y-%m-%dT%H:%M:%SZ").replace(tzinfo=timezone.utc)
    arizona_to_utc_offset = timedelta(hours=7)  # Arizona is UTC-7
    arizona_date_utc = utc_date - arizona_to_utc_offset
    return arizona_date_utc.date() == utc_date.date()

# Test the function
arizona_date_str = "2024-04-23"
utc_date_str = "2024-04-23T20:59:07Z"
print(are_dates_equal(arizona_date_str, utc_date_str))


2024-04-23 00:00:00
True


In [6]:
#!/home/austinmedina/DataLabMetrtics/Metrics/bin/python
#The above header, allows the file to be independently run by Cron jobs
import requests
import base64
from datetime import datetime, timedelta, date, timezone
import pytz
import psycopg2
import pandas as pd
import logging  

def getRefreshToken(zoomMeetingID, conn, cur):
    """
    Key Arguments:
    zoomMeetingID - The id associated with a zoom meeting that we will fetch a participant list for.
    conn - The connection to the Postgres database
    cur - The cursor to the Postgres database 

    Return:
    tokenList[0] - This is the refresh token needed to gain an access token

    Description:
    Fetches the refresh token for a specific zoom meeting ID, which is needed to fetch an access token.
    """
    cur.execute("""SELECT RefreshToken FROM ZoomRefreshTokens 
                WHERE ZoomMeetingID = %s
                """, (zoomMeetingID,))
    tokenList = cur.fetchone()

    conn.commit()
    return tokenList[0]

def getAccessToken(refreshToken, zoomMeetingID, conn, cur):
    """
    Key Arguments:
    refreshToken - The refresh token needed to get a new access token
    zoomMeetingID - The id associated with a zoom meeting that we will fetch a participant list for.
    conn - The connection to the Postgres database
    cur - The cursor to the Postgres database 

    Returns:
    accessToken - The token needed to make an API call to zoom for the participant list

    Description:
    Every meeting participant list can only be fetched with an access token. These access tokens however expire shortly after that are retrieved. The requests also return a refresh key which
    can be used to fetch a new access token. This function uses the refresh token to fetch a new access token, and stores the newly returned refresh token in the database.
    """
    cur.execute("""SELECT ElementValue FROM programvariables WHERE ElementName = 'zoom_client_id'""")
    tokenList = cur.fetchone()
    client_id = tokenList[0]

    cur.execute("""SELECT ElementValue FROM programvariables WHERE ElementName = 'zoom_client_secret'""")
    tokenList = cur.fetchone()
    client_secret = tokenList[0]
    conn.commit()

    auth_string = f"{client_id}:{client_secret}"
    encoded_auth = base64.b64encode(auth_string.encode()).decode()
    auth_header = f"Basic {encoded_auth}"

    data = {
        "refresh_token": refreshToken,
        "grant_type": "refresh_token",
    }

    headers = {
        "Authorization": auth_header,
        "Content-Type": "application/x-www-form-urlencoded"
    }

    # Send POST request
    response = requests.post("https://zoom.us/oauth/token", headers=headers, data=data)

    re = response.json()
    accessToken = re['access_token']
    refreshToken = re['refresh_token']

    cur.execute("""UPDATE ZoomRefreshTokens 
                SET RefreshToken = %s 
                WHERE ZoomMeetingID = %s
                """, (refreshToken, zoomMeetingID))
    conn.commit()

    return accessToken

def getParticipants(zoomMeetingID, conn, cur):
    """
    Fetches a list of meeting participants for a specific Zoom meeting ID, handling pagination.

    Key Arguments:
    zoomMeetingID - The ID associated with a Zoom meeting to fetch a participant list.
    conn - The connection to the Postgres database.
    cur - The cursor to the Postgres database.

    Returns:
    all_participants - A list of all participants, their email, join time, and leave time across all pages.
    """
    refreshToken = getRefreshToken(zoomMeetingID, conn, cur)
    accessToken = getAccessToken(refreshToken, zoomMeetingID, conn, cur)
    url = f'https://api.zoom.us/v2/past_meetings/{zoomMeetingID}/participants'
    
    headers = {
        'Authorization': f'Bearer {accessToken}',
        'page_size': '100'
    }

    all_participants = []
    next_page_token = ''
    
    while True:
        params = {'next_page_token': next_page_token} if next_page_token else {}
        response = requests.get(url, headers=headers, params=params)

        if response.status_code == 200:
            data = response.json()
            all_participants.extend(data.get('participants', []))
            next_page_token = data.get('next_page_token')
            if not next_page_token:
                break
        else:
            print(f'Error: {response.status_code}')
            break

    return all_participants

    
def uploadCheckIn(row, workshopID, conn, cur):
    """
    Checks in a user to a specific workshop if they are found in the particpants list

    Key Arguments:
    row - One person and their information from zoom such as email, join time, and leave time
    workshopID - The ID for the specific workshop they attended
    conn - The connection to the Postgres database
    cur - The cursor to the Postgres database 

    Return:
    No return, but updates the postgres database

    Description:
    The function takes in a participants information from the zoom api meeting participants list. Next a hash is created from the users email. We then find the participant in the database 
    or create a new entry. The users name is split into first and last, and then they are checked into the database by setting the checkedIn value to True.
    """
    #Fetch the supposed ID from the hash function
    cur.callproc('hashRegistree', (row.iloc[3].lower(),))
    hashedNum = cur.fetchone()[0]
    conn.commit()

    #See if the RegID is in the database already
    cur.execute("""SELECT RegID FROM registreeInfo WHERE RegID = %s""", (hashedNum,))
    match = cur.fetchall()
    conn.commit()

    #If the list isnt empty then a match was found so we will update their entry
    if (len(match) != 0):
        cur.execute("""
                    UPDATE RegistreeWorkshops
                    SET CheckedIn = TRUE
                    WHERE RegID = %s AND WorkshopId = %s
                    """, (hashedNum, workshopID))
        conn.commit()
    #If the list is empty, there is no registrant matching the workshop so we will need to create an entry for them
    else:
        logging.info("New registree created: " + str(hashedNum))
        #Create a registreeInfo entry for the person
        splitName = row.iloc[2]
        splitName = splitName.split(' ')
        splitName = [name.strip() for name in splitName]
        if (len(splitName) > 0):
            if (len(splitName) == 2):
                firstName = splitName[0]
                lastName = splitName[1]
            elif (len(splitName) > 2):
                firstName = splitName[0]
                lastName = ' '.join(splitName[1:])
            else:
                firstName = splitName[0]
                lastName = ''
        
            cur.execute("""
                        INSERT INTO registreeInfo (RegID, FirstName, LastName, NetID, Email, College, Department, Major, Recontact)
                        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
                        ON CONFLICT DO NOTHING;
                        """, (hashedNum, firstName, lastName, None, row.iloc[3].lower(), None, None, None, False))
            conn.commit()

            #Create an entry for the person and the specific workshop they attended
            cur.execute("""
                        INSERT INTO RegistreeWorkshops (RegID, WorkshopID, Registered, CheckedIn)
                        VALUES (%s, %s, %s, %s)
                        ON CONFLICT DO NOTHING
                        """, (hashedNum, workshopID, False, True))
            conn.commit()   

def uploadWithoutEmail(FirstName, LastName, workshopID, conn, cur):
    """
    Checks in a user based on first and last name, otherwise adds them to unknown people if they cannot be found.

    Key Arguments:
    FirstName - The first name as listed by the zoom api
    LastName - The last name as listed by the zoom api
    workshopID - The ID for the specific workshop they attended
    conn - The connection to the Postgres database
    cur - The cursor to the Postgres database 

    Return:
    No return but upload the person to the database

    Description:
    If no email is available for the participant (i.e. they signed in as a guest on zoom) we will attemp to upload them using their first and last name. First the script trys to find an existing
    registree with that first and last name. If its found, the function checks them in. If they arent found they are added to the unknown people table.
    """
    FirstName = FirstName.lower().capitalize()
    LastName = LastName.lower().capitalize()
    cur.execute("""SELECT RegID FROM registreeInfo 
                WHERE FirstName = %s AND LastName = %s
                """, (FirstName, LastName))
    match = cur.fetchone()
    conn.commit()

    if (match):
        cur.execute("""
                    UPDATE RegistreeWorkshops
                    SET CheckedIn = TRUE
                    WHERE RegID = %s AND WorkshopId = %s
                    """, (match[0], workshopID))
        conn.commit()
        logging.info("Checked in by name: " + FirstName + " " + LastName)
    else:
        cur.execute("""INSERT INTO UnknownPeople (FirstName, LastName, WorkshopID)
                    VALUES(%s, %s, %s)
                    ON CONFLICT DO NOTHING
                    """, (FirstName, LastName, workshopID))
        conn.commit()
        if cur.rowcount == 1:
            logging.info("Unknown Person Without Email Added: " + FirstName + " " + LastName)
        else:
            logging.debug("Unknown Person Already in Database: " + FirstName + " " + LastName)

def convert_to_arizona_time(utc_time_str):
    """
    Converts a string time in UTC to arizona time

    Key Arguments:
    utc_time_str - A timestamp in string format in UTC

    Returns:
    arizona_time.time() - The time in arizona time in datetime format
    """
    # Parse the UTC time string into a datetime object
    utc_time = datetime.strptime(utc_time_str, '%Y-%m-%dT%H:%M:%SZ')

    # Define UTC timezone
    utc_timezone = pytz.timezone('UTC')

    # Convert the UTC time to Arizona time
    arizona_timezone = pytz.timezone('America/Phoenix')
    arizona_time = utc_timezone.localize(utc_time).astimezone(arizona_timezone)

    return arizona_time.time()

def are_dates_equal(arizona_date, utc_date_str):
    """
    Checks if the date in arizona time of a workshop, is equal to a date in utc when the utc date is converted to arizona time. Used to tell if a workshop occured on the same day as the zoom

    Key Arguments:
    arizona_date - The date of a workshop in Date format
    utc_date_str - The date of a zoom meeting in utc time as a string

    Return:
    True or False if the dates are equal or not
    """
    utc_date = datetime.strptime(utc_date_str, "%Y-%m-%dT%H:%M:%SZ").replace(tzinfo=timezone.utc)
    arizona_to_utc_offset = timedelta(hours=7)  # Arizona is UTC-7
    arizona_date_utc = utc_date - arizona_to_utc_offset
    return arizona_date == arizona_date_utc.date()

def zoomProcess(conn, cur):
    """
    Matches the most recent zoom meeting with a workshop that occured today and that time, and checks in the participants

    Key Arguments:
    conn - The connection to the Postgres database
    cur - The cursor to the Postgres database 

    Description:
    The function first finds every workshop that occured today and their respective zoom meeting IDs. It then loops through every workshop and fetches the participants list for the associated
    zoom meeting ID. We then iterates through each person and compared their join and leave time with the start and end time of the workshop. If they joined and/or left within 15 minutes before 
    or after the start time or end time they will be checked in. Before checking in we verify they have an email, and if not we check them in without an email.
    """
    cur.execute("""
                SELECT workshops.WorkshopID, workshops.SeriesID, series.ZoomMeetingID, series.StartTime, series.EndTime, workshops.WorkshopDate FROM workshops
                JOIN series ON workshops.SeriesID = series.SeriesID
                WHERE workshops.WorkshopDate = now()::date;
                """)

    workshopList = cur.fetchall()
    conn.commit()

    today = date.today()

    for workshop in workshopList:
        participants = getParticipants(workshop[2], conn, cur) #ZoomMeetingID
        #participants = [{'id': 'sMyAcaaxTRqIxL1pmqO_tw', 'user_id': '16778240', 'name': 'Jeffrey Kent Gillan', 'user_email': 'jgillan@arizona.edu', 'join_time': '2024-09-03T20:25:39Z', 'leave_time': '2024-09-03T23:02:08Z', 'duration': 9389, 'registrant_id': '', 'failover': False, 'status': 'in_meeting', 'groupId': ''}, {'id': '', 'user_id': '16797696', 'name': 'UA DataLab', 'user_email': '', 'join_time': '2024-09-03T20:27:43Z', 'leave_time': '2024-09-03T20:27:58Z', 'duration': 15, 'registrant_id': '', 'failover': False, 'status': 'in_waiting_room', 'groupId': ''}, {'id': '', 'user_id': '16798720', 'name': 'UA DataLab', 'user_email': '', 'join_time': '2024-09-03T20:27:59Z', 'leave_time': '2024-09-03T23:02:07Z', 'duration': 9248, 'registrant_id': '', 'failover': False, 'status': 'in_meeting', 'groupId': ''}, {'id': '2M9An2omS9KPM8Rxz2lwfw', 'user_id': '16799744', 'name': 'Michele Cosi', 'user_email': 'cosi@arizona.edu', 'join_time': '2024-09-03T20:31:41Z', 'leave_time': '2024-09-03T23:02:08Z', 'duration': 9027, 'registrant_id': '', 'failover': False, 'status': 'in_meeting', 'groupId': ''}, {'id': '', 'user_id': '16819200', 'name': 'Sandeep Kulkarni', 'user_email': '', 'join_time': '2024-09-03T20:52:29Z', 'leave_time': '2024-09-03T20:53:13Z', 'duration': 44, 'registrant_id': '', 'failover': False, 'status': 'in_waiting_room', 'groupId': ''}, {'id': 'yXxO1qhiQkmJy4b80J8xAQ', 'user_id': '16820224', 'name': 'Carlos Lizárraga', 'user_email': 'clizarraga@arizona.edu', 'join_time': '2024-09-03T20:53:03Z', 'leave_time': '2024-09-03T21:27:21Z', 'duration': 2058, 'registrant_id': '', 'failover': False, 'status': 'in_meeting', 'groupId': ''}, {'id': '', 'user_id': '16821248', 'name': 'Sandeep Kulkarni', 'user_email': '', 'join_time': '2024-09-03T20:53:13Z', 'leave_time': '2024-09-03T21:53:31Z', 'duration': 3618, 'registrant_id': '', 'failover': False, 'status': 'in_meeting', 'groupId': ''}, {'id': 'uoTA-PPPScmSAMRObdoQig', 'user_id': '16822272', 'name': 'Tyson Swetnam', 'user_email': 'tswetnam@arizona.edu', 'join_time': '2024-09-03T20:53:51Z', 'leave_time': '2024-09-03T20:54:59Z', 'duration': 68, 'registrant_id': '', 'failover': False, 'status': 'in_waiting_room', 'groupId': ''}, {'id': '', 'user_id': '16823296', 'name': 'Eduardo Martinez', 'user_email': '', 'join_time': '2024-09-03T20:54:13Z', 'leave_time': '2024-09-03T20:55:04Z', 'duration': 51, 'registrant_id': '', 'failover': False, 'status': 'in_waiting_room', 'groupId': ''}, {'id': 'uoTA-PPPScmSAMRObdoQig', 'user_id': '16824320', 'name': 'Tyson Swetnam', 'user_email': 'tswetnam@arizona.edu', 'join_time': '2024-09-03T20:55:00Z', 'leave_time': '2024-09-03T21:53:19Z', 'duration': 3499, 'registrant_id': '', 'failover': False, 'status': 'in_meeting', 'groupId': ''}, {'id': '', 'user_id': '16825344', 'name': 'Eduardo Martinez', 'user_email': '', 'join_time': '2024-09-03T20:55:05Z', 'leave_time': '2024-09-03T21:53:24Z', 'duration': 3499, 'registrant_id': '', 'failover': False, 'status': 'in_meeting', 'groupId': ''}, {'id': 'qGebtr02SM-Q6CvMjYvn2A', 'user_id': '16826368', 'name': 'Dessa', 'user_email': 'dakeys@arizona.edu', 'join_time': '2024-09-03T20:56:03Z', 'leave_time': '2024-09-03T20:56:10Z', 'duration': 7, 'registrant_id': '', 'failover': False, 'status': 'in_waiting_room', 'groupId': 'h7f4ijsPQFGoJNQT3CfhrA'}, {'id': 'qGebtr02SM-Q6CvMjYvn2A', 'user_id': '16827392', 'name': 'Dessa', 'user_email': 'dakeys@arizona.edu', 'join_time': '2024-09-03T20:56:11Z', 'leave_time': '2024-09-03T21:53:38Z', 'duration': 3447, 'registrant_id': '', 'failover': False, 'status': 'in_meeting', 'groupId': 'h7f4ijsPQFGoJNQT3CfhrA'}, {'id': 'NsbTcPOoT_qDdx3tDdPQ8w', 'user_id': '16830464', 'name': 'Zach Winston', 'user_email': 'winston@arizona.edu', 'join_time': '2024-09-03T20:56:16Z', 'leave_time': '2024-09-03T20:56:27Z', 'duration': 11, 'registrant_id': '', 'failover': False, 'status': 'in_waiting_room', 'groupId': 'o4RhpwgjQNuSlfp_BzXIog'}, {'id': 'NsbTcPOoT_qDdx3tDdPQ8w', 'user_id': '16831488', 'name': 'Zach Winston', 'user_email': 'winston@arizona.edu', 'join_time': '2024-09-03T20:56:27Z', 'leave_time': '2024-09-03T21:24:20Z', 'duration': 1673, 'registrant_id': '', 'failover': False, 'status': 'in_meeting', 'groupId': 'o4RhpwgjQNuSlfp_BzXIog'}, {'id': '', 'user_id': '16832512', 'name': 'Yash Vijay Salokhe', 'user_email': '', 'join_time': '2024-09-03T20:57:25Z', 'leave_time': '2024-09-03T20:57:33Z', 'duration': 8, 'registrant_id': '', 'failover': False, 'status': 'in_waiting_room', 'groupId': ''}, {'id': '', 'user_id': '16833536', 'name': 'Yash Vijay Salokhe', 'user_email': '', 'join_time': '2024-09-03T20:57:33Z', 'leave_time': '2024-09-03T21:52:38Z', 'duration': 3305, 'registrant_id': '', 'failover': False, 'status': 'in_meeting', 'groupId': ''}, {'id': '', 'user_id': '16834560', 'name': 'Anirudh K', 'user_email': '', 'join_time': '2024-09-03T20:57:50Z', 'leave_time': '2024-09-03T20:58:20Z', 'duration': 30, 'registrant_id': '', 'failover': False, 'status': 'in_waiting_room', 'groupId': ''}, {'id': 'hBhR83maRx2z2O9ymxBrkw', 'user_id': '16835584', 'name': 'SAURABH KAUSHIK', 'user_email': 'skaushik@arizona.edu', 'join_time': '2024-09-03T20:58:07Z', 'leave_time': '2024-09-03T20:58:37Z', 'duration': 30, 'registrant_id': '', 'failover': False, 'status': 'in_waiting_room', 'groupId': ''}, {'id': '', 'user_id': '16836608', 'name': 'Dustin Curley', 'user_email': '', 'join_time': '2024-09-03T20:58:25Z', 'leave_time': '2024-09-03T20:58:39Z', 'duration': 14, 'registrant_id': '', 'failover': False, 'status': 'in_waiting_room', 'groupId': ''}, {'id': 'hBhR83maRx2z2O9ymxBrkw', 'user_id': '16837632', 'name': 'SAURABH KAUSHIK', 'user_email': 'skaushik@arizona.edu', 'join_time': '2024-09-03T20:58:38Z', 'leave_time': '2024-09-03T21:53:18Z', 'duration': 3280, 'registrant_id': '', 'failover': False, 'status': 'in_meeting', 'groupId': ''}, {'id': '', 'user_id': '16838656', 'name': 'Dustin Curley', 'user_email': '', 'join_time': '2024-09-03T20:58:39Z', 'leave_time': '2024-09-03T21:53:23Z', 'duration': 3284, 'registrant_id': '', 'failover': False, 'status': 'in_meeting', 'groupId': ''}, {'id': 'O89dEpRiSzKAnNR_ltqaCg', 'user_id': '16839680', 'name': 'Eduardo Jimenez', 'user_email': 'eduardojh@arizona.edu', 'join_time': '2024-09-03T20:58:51Z', 'leave_time': '2024-09-03T20:58:59Z', 'duration': 8, 'registrant_id': '', 'failover': False, 'status': 'in_waiting_room', 'groupId': 'o4RhpwgjQNuSlfp_BzXIog'}, {'id': 'Qe66mrfFQPqBc8BYDJtUTA', 'user_id': '16840704', 'name': 'Matthew Hall Moelling', 'user_email': 'mhmoelling@arizona.edu', 'join_time': '2024-09-03T20:58:54Z', 'leave_time': '2024-09-03T20:59:04Z', 'duration': 10, 'registrant_id': '', 'failover': False, 'status': 'in_waiting_room', 'groupId': ''}, {'id': 'O89dEpRiSzKAnNR_ltqaCg', 'user_id': '16841728', 'name': 'Eduardo Jimenez', 'user_email': 'eduardojh@arizona.edu', 'join_time': '2024-09-03T20:58:59Z', 'leave_time': '2024-09-03T21:53:53Z', 'duration': 3294, 'registrant_id': '', 'failover': False, 'status': 'in_meeting', 'groupId': 'o4RhpwgjQNuSlfp_BzXIog'}, {'id': 'Qe66mrfFQPqBc8BYDJtUTA', 'user_id': '16842752', 'name': 'Matthew Hall Moelling', 'user_email': 'mhmoelling@arizona.edu', 'join_time': '2024-09-03T20:59:05Z', 'leave_time': '2024-09-03T21:00:45Z', 'duration': 100, 'registrant_id': '', 'failover': False, 'status': 'in_meeting', 'groupId': ''}, {'id': '3pVAbp9aSaa7yNhvCKFq8w', 'user_id': '16843776', 'name': 'Andrew Corrales', 'user_email': 'andrewicorrales@arizona.edu', 'join_time': '2024-09-03T20:59:19Z', 'leave_time': '2024-09-03T20:59:37Z', 'duration': 18, 'registrant_id': '', 'failover': False, 'status': 'in_waiting_room', 'groupId': 'o4RhpwgjQNuSlfp_BzXIog'}, {'id': '5Shc802_TWyERUEchqUBhw', 'user_id': '16844800', 'name': 'Jeremy Weiss', 'user_email': 'jlweiss@arizona.edu', 'join_time': '2024-09-03T20:59:38Z', 'leave_time': '2024-09-03T20:59:45Z', 'duration': 7, 'registrant_id': '', 'failover': False, 'status': 'in_waiting_room', 'groupId': 'o4RhpwgjQNuSlfp_BzXIog'}, {'id': '3pVAbp9aSaa7yNhvCKFq8w', 'user_id': '16845824', 'name': 'Andrew Corrales', 'user_email': 'andrewicorrales@arizona.edu', 'join_time': '2024-09-03T20:59:38Z', 'leave_time': '2024-09-03T21:53:26Z', 'duration': 3228, 'registrant_id': '', 'failover': False, 'status': 'in_meeting', 'groupId': 'o4RhpwgjQNuSlfp_BzXIog'}, {'id': '5Shc802_TWyERUEchqUBhw', 'user_id': '16846848', 'name': 'Jeremy Weiss', 'user_email': 'jlweiss@arizona.edu', 'join_time': '2024-09-03T20:59:45Z', 'leave_time': '2024-09-03T21:53:20Z', 'duration': 3215, 'registrant_id': '', 'failover': False, 'status': 'in_meeting', 'groupId': 'o4RhpwgjQNuSlfp_BzXIog'}]
        if participants:
            #logging.info(participants)

            for person in participants:
                #Get the times the person was in the zoom
                userJoin = convert_to_arizona_time(person['join_time'])
                userLeave = convert_to_arizona_time(person['leave_time'])

                #Find the difference between when they joined and when they left. The combine is needed since you cannot subtract datetime.time objects
                joinDifference = abs(datetime.combine(today, userJoin) - datetime.combine(today, workshop[3]))
                leaveDifference = abs(datetime.combine(today, userLeave) - datetime.combine(today, workshop[4]))

                #Verify the dates are equal
                datesEqual = are_dates_equal(workshop[5], person['join_time'])

                #If they joined within 15 minutes of the start time or left within 15 minutes of the end time we will check them in
                if (( joinDifference < timedelta(minutes=15) ) or ( leaveDifference < timedelta(minutes=15) )) and (datesEqual):
                    if person['user_email']:
                        uploadCheckIn(pd.Series(person), workshop[0], conn, cur)
                        logging.info("Checked in: " + person['user_email'])
                        logging.info("--------------------------------------")
                    else:
                        splitName = person['name']
                        splitName = splitName.split(' ')
                        splitName = [name.strip() for name in splitName]
                        if (len(splitName) > 0):
                            if (len(splitName) == 2):
                                firstName = splitName[0]
                                lastName = splitName[1]
                            elif (len(splitName) > 2):
                                firstName = splitName[0]
                                lastName = ' '.join(splitName[1:])
                            else:
                                firstName = splitName[0]
                                lastName = ''

                            uploadWithoutEmail(firstName, lastName, workshop[0], conn, cur)
                            logging.info("--------------------------------------")
                        

if __name__ == '__main__':
    """
    The main function for the script. Configures logging and opens the connection to the database.
    """
    conn = psycopg2.connect(database = "DataLab", 
                            user = "postgres", 
                            host= 'localhost',
                            password = "",
                            port = 5432)

    cur = conn.cursor()

    logging.basicConfig(filename='/home/austinmedina/DataLabMetrtics/logging/zoomLogging.log', level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')
    logging.FileHandler('/home/austinmedina/DataLabMetrtics/logging/zoomLogging.log')

    logging.info("STARTING ZOOM CHECK IN PROCESS")
    logging.info("--------------------------------------")
    zoomProcess(conn, cur)
    logging.info("--------------------------------------")
    logging.info("ENDING ZOOM CHECK IN PROCESS")
    
    
    cur.close()
    conn.close()
